# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [122]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [82]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
# first row is link, useless; last two rows we know are empty, can skip now or dropna later
df = pd.read_csv(file_path, skiprows=1)[:-2] 
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')
#converts 'Late (31-120 days)' to 'high_risk', 'Late(16-30 days)' to 'high_risk', etc
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   loan_status                 68817 non-null  object 
 8   pymnt_plan                  68817 non-null  object 
 9   dti                         68817 non-null  float64
 10  delinq_2yrs                 68817 non-null  float64
 11  inq_last_6mths              68817 non-null  float64
 12  open_acc                    68817 non-null  float64
 13  pub_rec                     688

In [87]:
# Poking around and investigating the data

#df['verification_status'].unique()
#df['issue_d'].unique()
#df['home_ownership'].unique()
#df.groupby('loan_status').count()
#df["pymnt_plan"].unique()
#df.groupby("pymnt_plan").count()      #only a single one, can drop?
#df["initial_list_status"].unique()
#df["application_type"].unique()
#df["hardship_flag"].unique()
#df.groupby("hardship_flag").count()  #only a single one, can drop?
#df["loan_amnt"].duplicated().sum()
#df.duplicated(subset="hardship_flag",keep=False).sum()
#df.groupby("collection_recovery_fee").count()
#df["loan_status"].unique()

array(['Source Verified', 'Verified', 'Not Verified'], dtype=object)

In [85]:
# check if columns are single-valued, i.e. unimportant to the learning
i=0
singlevalcols = []
for col in columns:
    # df.duplicated() returns series of true/falses whether the value is repeated
    # adding .sum() adds up all the trues (1s)
    # with duplicated(keep="first") (default), each first encounter of a value is counted as duplicated
    # if a series is single-valued, the sum will be total length - 1
    #       because the value is not a duplicate at the first value
    # total length of this df is 68,817
    # if a series has at least N possible values, the sum can be total length - N
    #        because of N first encounters of each value
    # with duplicated(keep=False), even the first encounter of a value is counted as duplicated if there are others
    # messes with logic if values are from a set because total sum will always be total length
    
    if (df.duplicated(subset=col).sum() == 68816):
        #if df.groupby(col).count().iloc[0,0] == 68817: #unneeded with re-worked logic above
        i+=1
        singlevalcols.append(col)
        print(f"Column {col} is single-valued. The type is {df[col].dtype}. ({i})")

print(singlevalcols)

Column pymnt_plan is single-valued. The type is object. (1)
Column recoveries is single-valued. The type is float64. (2)
Column collection_recovery_fee is single-valued. The type is float64. (3)
Column policy_code is single-valued. The type is float64. (4)
Column acc_now_delinq is single-valued. The type is float64. (5)
Column num_tl_120dpd_2m is single-valued. The type is float64. (6)
Column num_tl_30dpd is single-valued. The type is float64. (7)
Column tax_liens is single-valued. The type is float64. (8)
Column hardship_flag is single-valued. The type is object. (9)
Column debt_settlement_flag is single-valued. The type is object. (10)
['pymnt_plan', 'recoveries', 'collection_recovery_fee', 'policy_code', 'acc_now_delinq', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'tax_liens', 'hardship_flag', 'debt_settlement_flag']


In [86]:
# Remove / drop the list of singlevalcols

df2 = df.drop(labels=singlevalcols, axis=1)

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 76 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   loan_status                 68817 non-null  object 
 8   dti                         68817 non-null  float64
 9   delinq_2yrs                 68817 non-null  float64
 10  inq_last_6mths              68817 non-null  float64
 11  open_acc                    68817 non-null  float64
 12  pub_rec                     68817 non-null  float64
 13  revol_bal                   688

In [110]:
# Transform categorical data to numerical data

from sklearn.preprocessing import LabelEncoder

# list of categorical features
categcols = ['home_ownership', 'verification_status', 'issue_d', 'initial_list_status', 'next_pymnt_d', 'application_type']

#len(categcols) # make sure there are 6

# transform categorical features
df3 = pd.get_dummies(df2, columns=categcols)

# remove spaces from column names to be safe
df3.rename(columns={'verification_status_Not Verified' : 'verification_status_Not_Verified',
                   'verification_status_Source Verified' : 'verification_status_Source_Verified',
                   'application_type_Joint App' : 'application_type_Joint_App'},
           inplace=True)

# fit and transform loan_status column
myencoder = LabelEncoder()
df3['loan_status'] = myencoder.fit_transform(df3['loan_status'])

df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            68817 non-null  float64
 1   int_rate                             68817 non-null  float64
 2   installment                          68817 non-null  float64
 3   annual_inc                           68817 non-null  float64
 4   loan_status                          68817 non-null  int32  
 5   dti                                  68817 non-null  float64
 6   delinq_2yrs                          68817 non-null  float64
 7   inq_last_6mths                       68817 non-null  float64
 8   open_acc                             68817 non-null  float64
 9   pub_rec                              68817 non-null  float64
 10  revol_bal                            68817 non-null  float64
 11  total_acc                   

In [118]:
# check encoding

display(df2.head(3))
display(df3.head(3))

###### IMPORTANT ######
# 0 : high_risk
# 1 : low_risk
###### IMPORTANT ######

df3['loan_status'].value_counts() # to double check it's the same below

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,dti,delinq_2yrs,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,27.24,0.0,...,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,20.23,0.0,...,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,24.26,0.0,...,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0


,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint_App
0,10500.0,0.1719,375.35,66000.0,1,27.24,0.0,0.0,8.0,0.0,...,0,0,0,1,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,1,20.23,0.0,0.0,17.0,1.0,...,1,0,0,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,1,24.26,0.0,0.0,8.0,0.0,...,1,0,0,1,0,1,0,1,1,0


1    68470
0      347
Name: loan_status, dtype: int64

# Split the Data into Training and Testing

In [112]:
# Create our features
X = df3.drop("loan_status", axis=1)


# Create our target
y = df3["loan_status"].copy()

In [115]:
X.describe()
#X.info()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint_App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.148001,0.371696,0.451066,0.177238,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.355104,0.483261,0.497603,0.381873,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [117]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [142]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(Counter(y_train))
print(Counter(y_test))

Counter({1: 51366, 0: 246})
Counter({1: 17104, 0: 101})


# No Sampling

Here, we perform a logistic regression with NO sampling corrections:

1. View the count of the target classes using `Counter` from the collections library. 
3. Train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1for each sampling algorithm to ensure consistency between tests

In [136]:
# create, fit/train, and test logistic regression model
noSampModel = LogisticRegression(random_state=1)
noSampModel.fit(X_train, y_train)

y_pred = noSampModel.predict(X_test)

In [148]:
# balanced accuracy score
print("Training Data Model score: ", noSampModel.score(X_train, y_train))
print("Testing Data Model score: ", noSampModel.score(X_test, y_test))
print("Testing Data Balanced score: ", balanced_accuracy_score(y_test, y_pred))

Training Data Model score:  0.9952142912500969
Testing Data Model score:  0.9941296134844522
Testing Data Balanced score:  0.5049212621215349


In [145]:
# confusion matrix
confusion_matrix(y_test, y_pred)

array([[    1,   100],
       [    1, 17103]], dtype=int64)

In [156]:
# imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.50      0.01      1.00      0.02      0.10      0.01       101
          1       0.99      1.00      0.01      1.00      0.10      0.01     17104

avg / total       0.99      0.99      0.02      0.99      0.10      0.01     17205



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [150]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_ROS_train, y_ROS_train = ros.fit_resample(X_train, y_train)

Counter(y_ROS_train)

Counter({1: 51366, 0: 51366})

In [151]:
# Train the Logistic Regression model using the resampled data
ROSmodel = LogisticRegression(random_state=1)
ROSmodel.fit(X_ROS_train, y_ROS_train)

y_ROS_pred = ROSmodel.predict(X_test)

In [153]:
# Calculated the balanced accuracy score
print("Training Data Model score: ", ROSmodel.score(X_ROS_train, y_ROS_train))
print("Testing Data Model score: ", ROSmodel.score(X_test, y_test))
print("Testing Data Balanced score: ", balanced_accuracy_score(y_test, y_ROS_pred))

Training Data Model score:  0.6429739516411634
Testing Data Model score:  0.6005231037489102
Testing Data Balanced score:  0.6711292709018329


In [154]:
# Display the confusion matrix
confusion_matrix(y_test, y_ROS_pred)

array([[   75,    26],
       [ 6847, 10257]], dtype=int64)

In [158]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_ROS_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.74      0.60      0.02      0.67      0.45       101
          1       1.00      0.60      0.74      0.75      0.67      0.44     17104

avg / total       0.99      0.60      0.74      0.74      0.67      0.44     17205



### SMOTE Oversampling

In [159]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_smote_train, y_smote_train = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train)

Counter(y_smote_train)

Counter({1: 51366, 0: 51366})

In [161]:
# Train the Logistic Regression model using the resampled data
smoteModel = LogisticRegression(random_state=1)
smoteModel.fit(X_smote_train, y_smote_train)

y_smote_pred = smoteModel.predict(X_test)

In [162]:
# Calculated the balanced accuracy score
print("Training Data Model score: ", smoteModel.score(X_smote_train, y_smote_train))
print("Testing Data Model score: ", smoteModel.score(X_test, y_test))
print("Testing Data Balanced score: ", balanced_accuracy_score(y_test, y_smote_pred))

Training Data Model score:  0.6527761554335553
Testing Data Model score:  0.6839872130194711
Testing Data Balanced score:  0.6589738721299632


In [163]:
# Display the confusion matrix
confusion_matrix(y_test, y_smote_pred)

array([[   64,    37],
       [ 5400, 11704]], dtype=int64)

In [164]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smote_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.63      0.68      0.02      0.66      0.43       101
          1       1.00      0.68      0.63      0.81      0.66      0.44     17104

avg / total       0.99      0.68      0.63      0.81      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [166]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_clustcent_train, y_clustcent_train = cc.fit_resample(X_train, y_train)

Counter(y_clustcent_train)

Counter({0: 246, 1: 246})

In [167]:
# Train the Logistic Regression model using the resampled data
clustcentModel = LogisticRegression(random_state=1)
clustcentModel.fit(X_clustcent_train, y_clustcent_train)

y_clustcent_pred = clustcentModel.predict(X_test)

In [168]:
# Calculated the balanced accuracy score
print("Training Data Model score: ", clustcentModel.score(X_clustcent_train, y_clustcent_train))
print("Testing Data Model score: ", clustcentModel.score(X_test, y_test))
print("Testing Data Balanced score: ", balanced_accuracy_score(y_test, y_clustcent_pred))

Training Data Model score:  0.6666666666666666
Testing Data Model score:  0.38668991572217376
Testing Data Balanced score:  0.5291324940492179


In [169]:
# Display the confusion matrix
confusion_matrix(y_test, y_clustcent_pred)

array([[   68,    33],
       [10519,  6585]], dtype=int64)

In [170]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_clustcent_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.67      0.38      0.01      0.51      0.27       101
          1       1.00      0.38      0.67      0.56      0.51      0.25     17104

avg / total       0.99      0.39      0.67      0.55      0.51      0.25     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [171]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_smoteen_train, y_smoteen_train = smote_enn.fit_resample(X_train, y_train)

Counter(y_smoteen_train)

Counter({0: 51361, 1: 46653})

In [172]:
# Train the Logistic Regression model using the resampled data
smoteenModel = LogisticRegression(random_state=1)
smoteenModel.fit(X_smoteen_train, y_smoteen_train)

y_smoteen_pred = smoteenModel.predict(X_test)

In [173]:
# Calculated the balanced accuracy score
print("Training Data Model score: ", smoteenModel.score(X_smoteen_train, y_smoteen_train))
print("Testing Data Model score: ", smoteenModel.score(X_test, y_test))
print("Testing Data Balanced score: ", balanced_accuracy_score(y_test, y_smoteen_pred))

Training Data Model score:  0.6530393617238354
Testing Data Model score:  0.569195001453066
Testing Data Balanced score:  0.640608936361363


In [174]:
# Display the confusion matrix
confusion_matrix(y_test, y_smoteen_pred)

array([[  72,   29],
       [7383, 9721]], dtype=int64)

In [175]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smoteen_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.71      0.57      0.02      0.64      0.41       101
          1       1.00      0.57      0.71      0.72      0.64      0.40     17104

avg / total       0.99      0.57      0.71      0.72      0.64      0.40     17205

